In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler


from keras.models import Sequential
from keras.layers import Dense, Activation
import keras
import tensorflow as tf

import properscoring as prscore

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [3]:
def get_feature_gain():
    target = ["generation"]
    features = ['Humidity', 'WindSpeed', 'Temp', 'CloudCover', 'Rain', 'SolarIrradiation', 
                'yearSin', 'yearCos', 'daySin', 'dayCos','monthSin', 'monthCos', 'hourSin', 'hourCos']
    #train = pd.read_csv("gs://hiroki_storage/pv/energies/code_energies/train.csv")
    df = pd.read_csv("/home/jupyter/code/data_energies.csv")
    train = df[df.month!=6]
    train_x,train_y = train[features],train[target]
    model = RandomForestRegressor(n_estimators=10, random_state=71)
    model.fit(train_x, train_y)
    feature_gain = model.feature_importances_
    feature_gain = pd.DataFrame(feature_gain,index=features,columns=["gain"]).sort_values(ascending=False,by="gain")
    return feature_gain

In [4]:
def train_test(day,use_col):
    df = pd.read_csv("/home/jupyter/code/data_energies.csv")
    train,test = df[df.month!=6],df[df.month==6]
    
    train = pd.concat([train,test[test.day<day]],axis=0)
    test= test[test.day==day]

    train_x,test_x=train[use_col],test[use_col]
    train_y = np.hstack([train[target],train[target]])
    test_y = np.hstack([test[target],test[target]])

    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)

    return train_x,test_x,train_y,test_y

In [5]:
def qd_test(y_true, y_pred):
    y_true = y_true[:,0]
    y_u = y_pred[:,0]
    y_l = y_pred[:,1]
    
    K_HU = tf.maximum(0.,tf.sign(y_u - y_true))
    K_HL = tf.maximum(0.,tf.sign(y_true - y_l))
    K_H = tf.multiply(K_HU, K_HL)
    
    K_SU = tf.sigmoid(soften_ * (y_u - y_true))
    K_SL = tf.sigmoid(soften_ * (y_true - y_l))
    K_S = tf.multiply(K_SU, K_SL)
    
    MPIW_c = tf.reduce_sum(tf.multiply((y_u - y_l),K_H))/tf.reduce_sum(K_H)
    PICP_S = tf.reduce_mean(K_S)    
    Loss_S = MPIW_c + lambda_ * n_ / (alpha_*(1-alpha_)) * ((tf.maximum(0.,(1-alpha_) - PICP_S))**2)
    
    return Loss_S,PICP_S,MPIW_c

def qd_objective(y_true, y_pred):
    y_true = y_true[:,0]
    y_u = y_pred[:,0]
    y_l = y_pred[:,1]
    
    K_HU = tf.maximum(0.,tf.sign(y_u - y_true))
    K_HL = tf.maximum(0.,tf.sign(y_true - y_l))
    K_H = tf.multiply(K_HU, K_HL)
    
    K_SU = tf.sigmoid(soften_ * (y_u - y_true))
    K_SL = tf.sigmoid(soften_ * (y_true - y_l))
    K_S = tf.multiply(K_SU, K_SL)
    
    MPIW_c = tf.reduce_sum(tf.multiply((y_u - y_l),K_H))/tf.reduce_sum(K_H)
    PICP_S = tf.reduce_mean(K_S)
    
    Loss_S = MPIW_c + lambda_ * n_ / (alpha_*(1-alpha_)) * ((tf.maximum(0.,(1-alpha_) - PICP_S))**2)
    
    return Loss_S

def get_LUBE(number_of_features,LR,BETA):
    model = Sequential()
    model.add(Dense(100, input_dim=number_of_features, activation='relu',
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.2)))
    model.add(Dense(2, activation='linear',
                    kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.3), 
                    bias_initializer=keras.initializers.Constant(value=[3.,-3.]))) 
    opt = tf.keras.optimizers.Adam(lr=LR, beta_1=BETA)
    model.compile(loss=qd_objective, optimizer=opt)
    return model

In [6]:
def quantile_regression_result(LOWER_ALPHA = 0.025,UPPER_ALPHA = 0.975,SEED = 0,Time = 1,features_counts = 1,\
                               lr=0.05, m_tr=3, m_le=7, n_e=500):
    
    df = pd.read_csv("/home/jupyter/code/data_energies.csv")

    train,test = df[df.month!=6],df[(df.month==6)&(df.day<15)]
    all_features = ['SolarIrradiation', 'hourSin', 'hourCos', 'yearCos', 'CloudCover',
       'yearSin', 'Humidity', 'Temp', 'monthCos', 'WindSpeed', 'daySin',
       'dayCos', 'monthSin', 'Rain']
    target = ["generation"]  
    
    result = pd.DataFrame()
    pred = pd.DataFrame()
    for i in range(Time):
        features = all_features[:features_counts]
        X_train,y_train = train[features],train[target]
        X_test,y_test = test[features],test[target]

        lower_model = GradientBoostingRegressor(loss="quantile",                   
                                                alpha=LOWER_ALPHA,
                                                random_state=SEED+100*i,
                                                learning_rate=lr,
                                                max_depth=m_tr,
                                                min_samples_leaf=m_le, 
                                                n_estimators=n_e,
                                                verbose=-1)
        
        upper_model = GradientBoostingRegressor(loss="quantile",
                                                alpha=UPPER_ALPHA,
                                                random_state=SEED+100*i,
                                                learning_rate=lr,
                                                max_depth=m_tr,
                                                min_samples_leaf=m_le, 
                                                n_estimators=n_e,
                                                verbose=-1)


        lower_model.fit(X_train, y_train)
        upper_model.fit(X_train, y_train)
        # Record actual values on test set
        predictions = pd.DataFrame(y_test)
        # Predict
        predictions['lower'] = lower_model.predict(X_test)
        predictions['upper'] = upper_model.predict(X_test)


        predictions["PICP"] = 0
        predictions.loc[(predictions.upper>=predictions.generation) & (predictions.generation>=predictions.lower),"PICP"] = 1
        predictions["MPIW"] = predictions["upper"] - predictions["lower"]
        predictions['day'] = test["day"]


        predictions["mean"] = (predictions["lower"]+predictions["upper"])/2
        predictions["std"] = ((predictions["lower"]+predictions["upper"])/2 - predictions["lower"])/1.96

        crps = []
        for i in range(predictions.shape[0]):
            d = predictions.iloc[i]
            C = prscore.crps_gaussian(d["generation"], mu=d["mean"], sig=d["std"])
            CRPS = C.mean()
            crps.append(CRPS)
        predictions["crps"] = crps
        predictions["features"] = features_counts
        pred = pred.append(predictions)

        CRPS,MPIW,PICP,Day = [],[],[],[]
        for day,group in predictions.groupby("day"):
            m = group[group.PICP>0].MPIW.mean()
            p = group.PICP.mean()
            c = group.crps.mean()
            MPIW.append(m)
            PICP.append(p)
            CRPS.append(c)
            Day.append(day)
        Loss = MPIW +  5/(0.05*0.95)*np.where(np.array([0]*14)>np.array([0.95]*14) -np.array(PICP),np.array([0]*14) ,np.array([0.95]*14) -np.array(PICP))

        scores = pd.DataFrame(list(zip(Day,MPIW,PICP,CRPS)),columns=["day","MPIW","PICP","CRPS"])
        scores["features"] = features_counts
        result = result.append(scores)
    return pred,result

In [7]:
alpha_ = 0.05
soften_ = 160
n_lambda = 5
EPOCHS = 3000
LR = 0.001
BETA = 0.01
Time = 1
SEED = 2021
number_of_features = 1
target = ["generation"]
features = ['Humidity', 'WindSpeed', 'Temp', 'CloudCover', 'Rain', 'SolarIrradiation', 
            'yearSin', 'yearCos', 'daySin', 'dayCos','monthSin', 'monthCos', 'hourSin', 'hourCos']

seed_everything(SEED)
feature_gain = get_feature_gain()


result = pd.DataFrame(columns=["number_of_features","day","Loss","PICP","MPIW"])
pred = pd.DataFrame(columns=["number_of_features","day","upper","lower"])
for i in range(Time):
    for day in range(1,15):
        pred_ = pd.DataFrame(columns=["number_of_features","day","upper","lower"])
        use_col = feature_gain.index[:number_of_features]
        X_train,X_test,y_train,y_test = train_test(day,use_col)

        n_= y_train.shape[0]
        lambda_ = n_lambda*1/n_ 

        model = get_LUBE(number_of_features,LR,BETA)

        history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=n_, verbose=0,  validation_split=0.)
        y_pred = model.predict(X_test, verbose=0)

        pred_[["upper","lower"]] = y_pred
        pred_[["number_of_features","day"]] = number_of_features,day
        pred_["Time"] = i
        pred = pd.concat([pred,pred_],axis=0)
        Loss_S_,PICP_S_,MPIW_c_ = qd_test(y_test,y_pred)
        scores = pd.DataFrame([Loss_S_,PICP_S_,MPIW_c_],index=["Loss","PICP","MPIW"]).T
        scores[["number_of_features","day"]] = number_of_features,day
        scores["Time"] = i
        result = pd.concat([result,scores],axis=0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
2022-07-12 02:32:57.947153: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
